In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

# Load Orders Dataset
orders = pd.read_csv("/content/Orders.csv")

# Load Trades Dataset with Corrected Column Names
trades = pd.read_csv("/content/Trades.CSV")

# ✅ Clean Column Names (Fix Spacing & Capitalization Issues)
orders.columns = orders.columns.str.strip().str.upper()
trades.columns = trades.columns.str.strip().str.upper()

# ✅ Ensure 'SCRIP_CODE' Exists in Both Datasets
if "SCRIP_CODE" not in orders.columns or "SCRIP_CODE" not in trades.columns:
    raise ValueError("SCRIP_CODE column is missing in one of the datasets!")

# ✅ Convert Order and Trade IDs to Strings to Avoid Scientific Notation Issues
orders["ORDER_ID"] = orders["ORDER_ID"].astype(str)
trades["BUY_ORDER_ID"] = trades["BUY_ORDER_ID"].astype(str)
trades["SELL_ORDER_ID"] = trades["SELL_ORDER_ID"].astype(str)

# ✅ Convert Date and Time Columns to Proper Format
orders["ORDER_TIMESTAMP"] = pd.to_datetime(orders["ORDER_DATE"] + " " + orders["ORDER_TIME"], errors="coerce")
trades["TRADE_TIMESTAMP"] = pd.to_datetime(trades["TRADE_DATE"] + " " + trades["TRADE_TIME"], errors="coerce")

# ✅ Selecting Key Features for Clustering
features = ["RATE", "QUANTITY", "TRADE_RATE", "TRADE_QUANTITY", "TRADE_VALUE"]

# ✅ Merge Orders and Trades on 'SCRIP_CODE'
merged = pd.merge(orders, trades, on="SCRIP_CODE", how="inner")

# ✅ Handle Missing Values
data = merged[features].dropna()

# ✅ Standardizing the Data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# ✅ Applying DBSCAN
dbscan = DBSCAN(eps=1.5, min_samples=10)
clusters = dbscan.fit_predict(data_scaled)

# ✅ Assign Cluster Labels
merged["Cluster"] = clusters

# ✅ Identifying Anomalies (Noise Points)
anomalies = merged[merged["Cluster"] == -1]

# ✅ Visualization - 2D Scatter Plot
plt.figure(figsize=(12, 6))
sns.scatterplot(data=merged, x="TRADE_QUANTITY", y="TRADE_VALUE", hue="Cluster", palette="viridis", alpha=0.7)
plt.title("DBSCAN Clustering for Fraud Detection")
plt.xlabel("Trade Quantity")
plt.ylabel("Trade Value")
plt.legend(title="Cluster")
plt.show()

# ✅ Interactive 3D Plot
fig = px.scatter_3d(merged, x="RATE", y="TRADE_QUANTITY", z="TRADE_VALUE",
                     color=merged["Cluster"].astype(str),
                     title="DBSCAN 3D Visualization",
                     opacity=0.7)
fig.show()

# ✅ Save Fraudulent Trades
anomalies.to_csv("Detected_Fraudulent_Trades.csv", index=False)
print(f"✅ Total Fraudulent Transactions Detected: {len(anomalies)}")


<ipython-input-4-3352c7a5ba59>:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  orders["ORDER_TIMESTAMP"] = pd.to_datetime(orders["ORDER_DATE"] + " " + orders["ORDER_TIME"], errors="coerce")
<ipython-input-4-3352c7a5ba59>:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trades["TRADE_TIMESTAMP"] = pd.to_datetime(trades["TRADE_DATE"] + " " + trades["TRADE_TIME"], errors="coerce")
